In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import csv
import pathlib


f = open('advice.dat', 'rb')
adv = pickle.load(f)

adv.reverse()



In [ ]:
def findFile(ticker):
    ticker = ticker.lower();
    for p in pathlib.Path('us').glob(f'**/{ticker}.txt') :
        return str(p);
    
    return ""

def r2c(value):
    return float(int(value*100+0.5))/100

def scanFile(fileName,enter_price,exit_price,start_date, end_date) :
    f = open(fileName, newline='')
    reader = csv.reader(f, delimiter=',', quotechar='|')
    
    is_title = True
    
    real_enter_price = 0.0
    real_exit_price = 0.0
    enter_find = False
    exit_find = False
    real_enter_date = None
    real_exit_date = None
    
    for row in reader:
        if is_title:
            is_title = False;
            continue;
            
        date = datetime.datetime.strptime(row[2],"%Y%m%d")
        o=float(row[4])
        h=float(row[5])
        l=float(row[6])
        c=float(row[7])
        
        
        if not enter_find:
            if start_date <= date and l<=enter_price:
                real_enter_price = min(o,enter_price)
                real_exit_price = real_enter_price
                real_enter_date = date
                enter_find = True;
        elif h>=exit_price:
            real_exit_price = max(o,exit_price)
            real_exit_date = date
            exit_find = True
            break
        else:
            real_exit_price = (h+l)/2
            real_exit_date = date
            
            if end_date <= date:
                break

    f.close()
    
    real_enter_price = r2c(real_enter_price)
    real_exit_price = r2c(real_exit_price)
    return [real_enter_price,real_exit_price,enter_find,exit_find,real_enter_date,real_exit_date]

class Res:
    def __init__(self,ticker,start_date,end_date,enter_price,exit_price,value,renter_price,rexit_price,enter_find,exit_find,renter_date,rexit_date):
        self.ticker=ticker
        self.start_date=start_date
        self.end_date=end_date
        self.enter_price=enter_price
        self.exit_price=exit_price
        self.value=value
        self.renter_price=renter_price
        self.rexit_price=rexit_price
        self.enter_find=enter_find
        self.exit_find=exit_find
        self.renter_date=renter_date
        self.rexit_date=rexit_date
    

In [ ]:
i=0
current_date = datetime.datetime.now()

results=[]

for a in adv:
    ticker = a[0]
    start_date = datetime.datetime.strptime(a[1],"%Y-%m-%d %H:%M")
    enter_price = float(a[2])
    exit_price = float(a[3])
    value = float(a[4].replace("%",""))
    
    
    duration = a[6]
    if duration.find(" months") == -1 and duration.find(" month") == -1:
        raise RuntimeError("expecting months")
    
    duration = duration.replace(" months","")
    duration = duration.replace(" month","")
    
    pos = duration.find("-")
    if pos != -1:
        duration = duration[pos+1:]
        
    duration = int(duration)
        
    end_date=start_date + datetime.timedelta(days=31*duration)
    
    #if end_date > current_date:
    #    continue;
    
    #print(a)
    #print (f'{i}: {ticker}: {enter_price} {exit_price} {start_date} == {end_date}')
    
    i=i+1
    
    fileName = findFile(ticker)
    if fileName == "":
        print(f"ticker {ticker} not found")
        continue;
    
    renter_price,rexit_price,enter_find,exit_find,renter_date,rexit_date = scanFile(fileName,enter_price,exit_price,start_date, end_date)
    
    ticker = a[0]
    start_date = datetime.datetime.strptime(a[1],"%Y-%m-%d %H:%M")
    enter_price = float(a[2])
    exit_price = float(a[3])
    value = float(a[4].replace("%",""))
    duration = int(duration)

    r=Res(ticker,start_date,end_date,enter_price,exit_price,value,renter_price,rexit_price,enter_find,
          exit_find,renter_date,rexit_date)
    
    results.append(r)
    

In [ ]:
total_in_money = 0
total_out_money = 0
total_fees = 0;
step_ammount=100

def fees(price,items_count):
    ret = price*items_count*0.5/100
    
    if items_count<100:
        ret+=1.2
    else:
        ret+=0.012*items_count
    
    return r2c(ret)


for r in results:
    if not r.enter_find:
        continue
        
    if r.end_date > current_date and not r.exit_find:
        continue;
        
    can_spend = r.value*step_ammount 
    
    items_count=int(can_spend/r.renter_price)

    if items_count == 0:
        continue

    in_money = items_count*r.renter_price
    out_money = items_count*r.rexit_price
    diff = out_money - in_money
    
    total_in_money=total_in_money+in_money
    total_out_money=total_out_money+out_money
    
    in_fees = fees(r.renter_price,items_count)
    out_fees = fees(r.rexit_price,items_count)
    
    total_fees = total_fees+in_fees+out_fees;
    
    ee=""
    if r.exit_find: ee='*'
        
    enter_date = r.renter_date.strftime('%Y-%m-%d')
    exit_date = r.rexit_date.strftime('%Y-%m-%d')
    
    print(f"{ee}{r.ticker}:({enter_date};{exit_date}) {r.renter_price}->{r.rexit_price} ({r.enter_price}->{r.exit_price}) diff={r2c(diff)} N={items_count} in_fees={in_fees} out_fees={out_fees}")

In [ ]:
pers=r2c((total_out_money-total_in_money-total_fees)/total_in_money*100)
print(f"in={r2c(total_in_money)} out={r2c(total_out_money)} fees={r2c(total_fees)} diff={r2c(total_out_money-total_in_money)} pers={pers}%")